<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/Introduction_to_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasification and localization

Localizing an object in a picture can be expressed as a regression task: to predict a bounding box around the object, a common approach is to
predict the horizontal and vertical coordinates of the object’s center, as well as its height and width. This means we have 4 numbers to predict. It does not require much change to the classification model, we just need to add a second dense output layer with 4 units, and it can be trained using the MSE loss.

The task of classifying and localizing a single object in an image is called *classification and localization*. The bounding boxes should be normalized so that the horizontal and vertical coordinates, as well as the height and width, all range from 0 to 1. 

The neural network now will output the above four numbers, plus the probability of class labels (four in an example with 4 classes for pedestrians, cars, bike and background). Therefore, the target label will be:

\begin{equation*}Y=
\begin{bmatrix}
           p_c \\
           b_x\\
           b_y\\
           b_h\\
           b_w\\
           c_1 \\
           c_2 \\
           c_3 
\end{bmatrix}
\end{equation*}

Where $p_c$, usually called *objectness* is the confidence of an object to be in the image. It responds to the question “Is there an object?” Instead, $c_1$, $c_2$, $c_3$, in case there is an object, tell if the object is part of class 1, 2 or 3. So, it tells us which object it is. Finally, $b_x$, $b_y$, $b_h$, $b_w$ identify the coordinates related to the bounding box around the detected object.
For example, if an image has a car, the target label will be:

\begin{equation*}Y=
\begin{bmatrix}
           1 \\
           b_x\\
           b_y\\
           b_h\\
           b_w\\
           0 \\
           1 \\
           0 
\end{bmatrix}
\end{equation*}

In case the network doesn’t detect an object, the output is simply:

\begin{equation*}Y=
\begin{bmatrix}
           0 \\
           ?\\
           ?\\
           ?\\
           ?\\
           ? \\
           ? \\
           ? 
\end{bmatrix}
\end{equation*}

Where the question marks are placed in the rest of the positions that don’t provide any meaning in this case. Technically the network will output big numbers or NaN in these positions.

**Note**: This technique is also used for *Landmarks detection*. In this case, the output will be even bigger since we ask the network to output the $x$ and $y$ coordinates of important points within an image. For example, think about an application for detecting key landmarks of a face. In this situation, we could identify points along the face that denote, for example, the corners of the eyes, the mouth, etc.


The MSE often works fairly well as a cost function to train the model, but it is not a great metric to evaluate how well the model can predict bounding boxes. The most common metric for this is the *Intersection over Union* (IoU): it is the area of overlap between the predicted bounding box and the target bounding box, divided by the area of their union (see Figure 14-23).

![texto alternativo](https://i.ibb.co/pv7k8Vh/IoU.png)

![texto alternativo](https://i.ibb.co/R0bgFsC/IoU2.png)

In `tf.keras`, it is implemented by the `tf.keras.metrics.MeanIoU` class.

What if the images contain multiple objects?

# Object detection


The task of classifying and localizing multiple objects in an image is called *object detection*. Until a few years ago, a common approach was to take a CNN that was trained to classify and locate a single object, then slide it across the image, as shown in Figure 14-24. 

![texto alternativo](https://i.ibb.co/pbWd6QL/sliding-CNN.png)

In this example, the image was chopped into a 6 $\times$ 8 grid, and we
show a CNN (the thick black rectangle) sliding across all 3 $\times$ 3 regions. When the CNN was looking at the top left of the image, it detected part of the left-most rose, and then it detected that same rose again when it was first shifted one step to the right. At the next step, it started detecting part of the top-most rose, and then it detected it again once it was shifted one more step to the right. You would then continue to slide the CNN through the whole image, looking at all 3 $\times$ 3 regions. Moreover, since objects can have varying sizes, you would also slide the CNN across regions of different sizes. For example, once you are done with the 3 $\times$ 3 regions, you might want to slide the CNN across all 4 $\times$ 4 regions as well.

This technique (called *sliding window detection*) is fairly straightforward, but as you can see it will detect the same object multiple times, at slightly different positions. Some post-processing will then be needed to get rid of all the unnecessary bounding boxes. A common approach for this is called *non-maximal suppression* or *non-max suppression*:



## Non-Maximal Suppression

- First, you need the *objectness* output to your CNN, to estimate the probability that an object is indeed present in the bounding box. It must use the sigmoid activation function and you can train it using the `"binary_crossentropy"` loss. Then just get rid of all the bounding boxes for which the objectness score is below some threshold: this will drop all the bounding boxes that don’t actually contain a flower.

- Second, find the bounding box with the highest objectness score, and get rid of all the other bounding boxes that overlap a lot with it (this is, with an IoU greater than 60%). For example, in Figure 14-24, the bounding box with the max objectness score is the thick bounding box over the top-most rose (the objectness score is represented by the thickness of the bounding boxes). The other bounding box over that same rose overlaps a lot with the max bounding box, so we will get rid of it.

- Third, repeat step two until there are no more bounding boxes to get rid of.

This simple approach to object detection works pretty well, but it requires running the CNN many times, so it is quite slow and it has a big computational cost. Fortunately, there is a much faster way to slide a CNN across an image: using a *Fully Convolutional Network*.

For a visual explanation of non-max suppression check this [video](https://www.youtube.com/watch?v=VAo84c1hQX8&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF&index=28).


## Fully Convolutional Networks (FCNs)

The idea of FCNs was first introduced in a [paper](https://arxiv.org/abs/1411.4038) by Jonathan Long in 2015, for semantic segmentation (the task of classifying every pixel in an image according to the class of the object it belongs to). They pointed out that you could replace the dense layers at the top of a CNN by convolutional layers. To understand this, let’s look at an example: suppose a dense layer with 400 neurons sits on top of a convolutional layer that outputs 16 feature maps, each of size 5 $\times$ 5 (this is the feature map size, not the kernel size).

![texto alternativo](https://i.ibb.co/LxW5bxw/fully-2.png)


Now let’s see what happens if we replace the dense layer with a convolution layer using 400 filters, each 5 $\times$ 5, and with VALID padding. This layer will output 400 feature maps, each 1 $\times$ 1 (since the kernel is exactly the size of the input feature maps and we are using VALID padding). In other words, it will output 400 numbers, just like the dense layer did, and if you look closely at the computations performed by a convolutional layer, you will notice that these numbers will be precisely the same as the dense layer produced. The only difference is that the dense layer’s output was a tensor of shape [batch size, 400] while the convolutional layer will output a tensor of shape [batch size, 1, 1, 400].

![texto alternativo](https://i.ibb.co/cc1zy5f/fcn.png)

**Note**: To convert a dense layer to a convolutional layer, the number of filters in the convolutional layer must be equal to the number of units in the dense layer, the filter size must be equal to the size of the input feature maps, and you must use VALID padding. The stride may be set to 1 or more, as we will see shortly.

Why is this important? Well, while a dense layer expects a specific input size (since it has one weight per input feature), a convolutional layer will happily process images of any size (however, it does expect its inputs to have a specific number of channels, since each kernel contains a different set of weights for each input channel). Since an FCN contains only convolutional layers (and pooling layers, which have the same property), it can be trained and executed on images of any size!

Now, to simplify the representation, let’s re-sketch the final network in 2D:

![texto alternativo](https://i.ibb.co/SJYGg6p/fully-1.png)

If our test image is of dimension 16$\times$16$\times$3 and we had to perform the “regular” sliding window we would have to create 4 different windows of size 14$\times$14$\times$3 out of the original test image and run each one through the ConvNet.

![texto alternativo](https://i.ibb.co/HnW6WN8/regular-sliding-windows.png)


This is computationally expensive and a lot of this computation is duplicative. We would like, instead, to have these four passes to share computation. So, with the convolutional implementation of sliding windows, we run the ConvNet, with the same parameters and same filters on the test image and this is what we get:

![texto alternativo](https://i.ibb.co/hcB8cXG/sliding-window-convolutionally.png)

Each of the 4 subsets of the output unit is essentially the result of running the ConvNet with a 14$\times$14$\times$3 region in the four positions on the initial 16$\times$16$\times$3 image.

This works on other examples too. Think about an input image of 28$\times$28$\times$3. Going through the network, we arrive at the final output of 8$\times$8$\times$400. In this one, each of the 8 subsets corresponds to running the 14$\times$14$\times$3 region 8 times with a slide of 2 in the original image.




Let's see another example. Suppose we already trained a CNN for flower classification and localization. It was trained on 224 $\times$ 224 images and it outputs 10 numbers: outputs 0 to 4 are sent through the softmax activation function, and this gives the class probabilities (one per class); output 5 is sent through the logistic activation function, and this gives the objectness score; outputs 6 to 9 do not use any activation function, and they represent the bounding box’s center coordinates, and its height and width. We can now convert its dense layers to convolutional layers. In fact, we don’t even need to retrain it, we can just copy the weights from the dense layers to the convolutional layers! Alternatively, we could have converted the CNN into an FCN before training.


Now suppose the last convolutional layer before the output layer (also called the bottleneck layer) outputs 7 $\times$ 7 feature maps when the network is fed a 224 $\times$ 224 image (see the left side of Figure 14-25).

![texto alternativo](https://i.ibb.co/PFfqwqK/FCN.png)

If we feed the FCN a 448 $\times$ 448 image (see the right side of Figure 14-25), the bottleneck layer will now output 14 $\times$ 14 feature maps (using SAME padding). Since the dense output layer was replaced by a convolutional layer using 10 filters of size 7 $\times$ 7, VALID padding, and stride 1, the output will be composed of 10 feature maps, each of size 8 × 8 (since 14 - 7 + 1 = 8). In other words, the FCN will process the whole image only once and it will output an 8 × 8 grid where each cell contains 10 numbers (5 class probabilities, 1 objectness score, and 4 bounding box coordinates). It’s exactly like taking the original CNN and sliding it across the image using 8 steps per row and 8 steps per column: to visualize this, imagine chopping the original image into a 14 $\times$ 14 grid, then sliding a 7 $\times$ 7 window across this grid: there will be 8 $\times$ 8 = 64 possible locations for the window, hence 8 $\times$ 8 predictions. However, the FCN approach is much more efficient, since the network only looks at the image once. In fact, *You Only Look Once* (YOLO) is the name of a very popular object detection architecture!

For a further explanation of FCN check this [video](https://www.youtube.com/watch?v=XdsmlBGOK-k&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF&index=26).

## Mean Average Precision

**Mean Average Precision (mAP)**: A very common metric used in object detection tasks is the *mean Average Precision* (mAP). To understand this metric, let’s go back to two classification metrics: precision and
recall. The higher the recall, the lower the precision. You can
visualize this in a Precision/Recall curve. To summarize this curve
into a single number, we could compute its Area Under the Curve (AUC). But note that the Precision/Recall curve may contain a few sections where precision actually goes up when recall increases, especially at low recall values. This is one of the motivations for the mAP metric.
Suppose the classifier has a 90% precision at 10% recall, but a 96% precision at 20% recall: there’s really no tradeoff here: it simply makes more sense to use the classifier at 20% recall rather than at 10% recall, as you will get both higher recall and higher precision. So instead of looking at the precision at 10% recall, we should really be looking at the maximum precision that the classifier can offer with at least 10% recall. It would be 96%, not 90%. So one way to get a fair idea of the model’s performance is to compute the maximum precision you can get with at least 0% recall, then 10% recall, 20%, and so on up to 100%, and then calculate the mean of these maximum
precisions. This is called the Average Precision (AP) metric. Now when there are more than 2 classes, we can compute the AP for each class, and then compute the mean AP (mAP). That’s it!

However, in object detection systems, there is an additional level of complexity: what if the system detected the correct class, but at the wrong location (i.e., the bounding box is completely off)? Surely we should not count this as a positive prediction. So one approach is to define an IOU threshold: for example, we may consider that a prediction is correct only if the IOU is greater than, say, 0.5, and the predicted class is correct. The corresponding mAP is generally noted mAP@$0.5$ (or mAP@50%, or sometimes just AP$_{50}$). In some competitions (such as the Pascal VOC challenge), this is what is done. In others (such as the COCO competition), the mAP is computed for different IOU thresholds (0.50, 0.55, 0.60, ..., 0.95), and the final metric is the mean of all these mAPs (noted AP@[.50:.95] or AP@[.50:0.05:.95]). Yes, that’s a mean mean average.


## Object detection algorithms


A modern object detector is usually composed of two parts, a backbone which is usually pre-trained on ImageNet (or in a similar dataset) and a head which is used to predict classes and bounding boxes of objects. The backbone refers to the network which takes as input the image and extracts the features maps. It could be VGG, ResNet, ResNeXt, etc. The head part is usually categorized into two kinds, i.e., one-stage object detector and two-stage object detector. One-stage object detectors takes only one forward propagation pass through the network to make the predictions and they are usually faster than two-stage detectors but they usually have less accuracy. The most representative one-stage object detectors are *You Only Look Once* ([YOLOv1](https://arxiv.org/abs/1506.02640), [YOLOv2](https://arxiv.org/abs/1612.08242), [YOLOv3](https://arxiv.org/abs/1804.02767), [YOLOv4](https://arxiv.org/abs/2004.10934)), *Single Shot MultiBox Detector* ([SSD](https://arxiv.org/abs/1512.02325)) ,and [RetinaNet](https://arxiv.org/abs/1708.02002). As for two-stage object detector, the most representative models are the [R-CNN](https://arxiv.org/abs/1311.2524) series (including [fast R-CNN](https://arxiv.org/abs/1504.08083), [faster R-CNN](https://arxiv.org/abs/1506.01497) and [R-FCN](https://arxiv.org/abs/1605.06409)). Object detectors developed in recent years often insert some layers between backbone and head, and these layers are usually used to collect feature maps from different stages. We can call it the *neck* of an object detector. Usually, a neck is composed of several bottom-up paths and several top-down paths. Networks equipped with this mechanism include *Feature Pyramid Network* ([FPN](https://arxiv.org/abs/1612.03144)) or *Path Aggregation Network* ([PAN](https://arxiv.org/abs/1803.01534)).


To sum up, object detectors tipically are compose of several components:

![texto alternativo](https://i.ibb.co/LN2vn7q/object-detection-flow.png)









The choice of detection system depends on many factors: speed, accuracy, available pretrained models, training time, complexity, etc. The papers contain tables of metrics, but there is quite a lot of variability in the testing environments, and the technologies evolve so fast that it is difficulty to make a fair comparison that will be useful for most people and remain valid for more than a few months.

# References

- [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

- [Course CNN deeplearning.ai.](https://www.youtube.com/watch?v=ArPaAX_PhIs&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF)


- [Learning non-maximum suppression](https://arxiv.org/abs/1705.02950)

- [Fully Convolutional Networks for Semantic Segmentation](https://arxiv.org/abs/1411.4038)